In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

In [5]:
class HWDDataset(Dataset):
    def __init__(self, file_path):
        self.data = []
        self.labels = []

        with open(file_path, 'r') as f:
            lines = f.read().strip().split("\n")
        
        sample = []
        for line in lines:
            line = line.strip()
            if len(line) == 1 and line.isdigit(): 
                if sample:
                    self.data.append(np.array(sample, dtype=np.float32)) 
                self.labels.append(int(line))
                sample = []
            else:
                sample.append([int(char) for char in line])

        if sample:
            self.data.append(np.array(sample, dtype=np.float32))

        self.data = [img.flatten() for img in self.data]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [6]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 16 * 16)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

def train_model(model, train_loader, criterion, optimizer, device):
    model.train()
    for images, labels in train_loader:
        images = images.view(-1, 1, 32, 32).to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

def evaluate_model(model, test_loader, device):
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.view(-1, 1, 32, 32).to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return all_preds, all_labels

In [8]:
if __name__ == "__main__":
    train_dataset = HWDDataset("C:\Dev\ANN\HWD_train.txt")
    test_dataset = HWDDataset("C:\Dev\ANN\HWD_test.txt")

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = CNN().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(10):
        train_model(model, train_loader, criterion, optimizer, device)
        print(f"Epoch {epoch+1}/10 completed")

    preds, labels = evaluate_model(model, test_loader, device)

    print("Confusion Matrix:")
    print(confusion_matrix(labels, preds))
    print("Classification Report:")
    print(classification_report(labels, preds))

Epoch 1/10 completed
Epoch 2/10 completed
Epoch 3/10 completed
Epoch 4/10 completed
Epoch 5/10 completed
Epoch 6/10 completed
Epoch 7/10 completed
Epoch 8/10 completed
Epoch 9/10 completed
Epoch 10/10 completed
Confusion Matrix:
[[178   0   0   0   0   0   0   0   0   0]
 [  0 182   0   0   0   0   0   0   0   0]
 [  0   1 175   0   0   0   0   1   0   0]
 [  1   0   1 175   0   1   0   2   3   0]
 [  0   2   0   0 176   0   0   0   2   1]
 [  0   0   0   1   0 180   0   0   0   1]
 [  1   0   0   0   1   0 178   0   1   0]
 [  0   0   0   0   0   4   0 170   1   4]
 [  0   4   0   0   1   0   0   0 164   5]
 [  0   1   0   2   0   1   0   0   2 174]]
Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       178
           1       0.96      1.00      0.98       182
           2       0.99      0.99      0.99       177
           3       0.98      0.96      0.97       183
           4       0.99      0.97      0.98   